In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 30.7 MB/s 
     |████████████████████████████████| 115 kB 52.8 MB/s 
     |████████████████████████████████| 212 kB 64.2 MB/s 
     |████████████████████████████████| 101 kB 10.6 MB/s 
     |████████████████████████████████| 127 kB 71.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, SpatialDropout1D, LSTM
from tensorflow.keras.layers import Bidirectional 
from tensorflow.keras.layers import Conv1D, MaxPooling1D 
from tensorflow.keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score, classification_report
from datasets import load_dataset
from tensorflow.keras.datasets import imdb
STOP_WORDS = ["went", "so", "the", "and", "a", "of", "from", "to", "not", "is", "in", "that", "this", "was", "as", "with", "for", "you", "are", "it"]

print(tf.__version__)

2.8.2


In [ ]:
# output directory name:
output_dir = 'model_output/LSTM/'

# training:
epochs = 4
batch_size = 128

# vector-space embedding: 
n_dim = 64 
max_review_length = 200 
pad_type = trunc_type = 'pre'
drop_embed = 0.2 

# convolutional layer architecture:
n_conv = 64  
k_conv = 3 
mp_size = 4

# LSTM layer architecture:
n_lstm = 64 
drop_lstm = 0.2

In [ ]:
vocab_length = 5000

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def tokenize_last(corpus):
  all_words = []
  for sent in corpus:
      tokenize_word = word_tokenize(sent)
      for word in tokenize_word:
          all_words.append(word)
  return all_words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
all_words = tokenize_last(dataset['train']['text'])

In [ ]:
unique_words = set(all_words)
print(len(unique_words))

124676


In [ ]:
from keras.preprocessing.text import one_hot

dataset = load_dataset("ag_news")
embedded_sentences_train = [one_hot(sent, vocab_length) for sent in dataset['train']['text']]
embedded_sentences_test = [one_hot(sent, vocab_length) for sent in dataset['test']['text']]

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
embedded_sentences_train[0]

[510,
 2975,
 3281,
 39,
 3096,
 1700,
 4752,
 2984,
 4607,
 4607,
 2972,
 772,
 510,
 1433,
 4777,
 4900,
 133,
 4888,
 1751,
 3771,
 3866,
 1449,
 2789]

In [ ]:
x_train = pad_sequences(embedded_sentences_train, 200, padding='post')
x_valid = pad_sequences(embedded_sentences_test, 200, padding='post')
x_train[0]

array([ 510, 2975, 3281,   39, 3096, 1700, 4752, 2984, 4607, 4607, 2972,
        772,  510, 1433, 4777, 4900,  133, 4888, 1751, 3771, 3866, 1449,
       2789,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [ ]:
y_train = np.array(dataset['train']['label'])
y_valid = np.array(dataset['test']['label'])

In [ ]:
n_dim = 200

model = Sequential()
model.add(Embedding(5000, n_dim, input_length=max_review_length)) 
model.add(SpatialDropout1D(drop_embed))
model.add(Conv1D(n_conv, k_conv, activation='relu'))
model.add(MaxPooling1D(mp_size))
model.add(Bidirectional(LSTM(n_lstm, dropout=drop_lstm)))
model.add(Dense(4, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 200)          1000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 200, 200)         0         
 ropout1D)                                                       
                                                                 
 conv1d (Conv1D)             (None, 198, 64)           38464     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 49, 64)           0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, 128)              66048     
 l)                                                              
                                                        

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [ ]:
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
x_valid.shape, y_valid.shape

((7600, 200), (7600,))

In [ ]:
x_train.shape, y_train.shape

((120000, 200), (120000,))

In [ ]:
# Batch size 128 returns error ( out of memory)
num_epochs = 4
model.fit(x_train, y_train, batch_size=16, epochs=num_epochs, validation_data=(x_valid, y_valid), verbose=1, callbacks=[modelcheckpoint])

Epoch 1/4
7500/7500 [==============================] - 102s 11ms/step - loss: 0.3816 - accuracy: 0.8644 - val_loss: 0.3173 - val_accuracy: 0.8871
Epoch 2/4
7500/7500 [==============================] - 84s 11ms/step - loss: 0.2730 - accuracy: 0.9068 - val_loss: 0.3034 - val_accuracy: 0.8968
Epoch 3/4
7500/7500 [==============================] - 84s 11ms/step - loss: 0.2312 - accuracy: 0.9200 - val_loss: 0.2945 - val_accuracy: 0.8982
Epoch 4/4
7500/7500 [==============================] - 83s 11ms/step - loss: 0.2022 - accuracy: 0.9305 - val_loss: 0.3208 - val_accuracy: 0.8953


In [ ]:
y_pred = model.predict(x_valid)
y_pred

In [ ]:
y_valid, y_pred

(array([2, 3, 3, ..., 1, 2, 2]),
 array([[2.7160976e-02, 1.9742409e-03, 7.3525363e-01, 2.3561114e-01],
        [1.1096754e-03, 5.2670486e-05, 1.0208582e-02, 9.8862898e-01],
        [3.2184616e-02, 3.1114617e-02, 2.5721874e-02, 9.1097891e-01],
        ...,
        [2.1224637e-03, 9.9761707e-01, 1.3717274e-04, 1.2332114e-04],
        [1.4208402e-01, 3.7851273e-03, 7.7934653e-01, 7.4784383e-02],
        [2.2439161e-02, 3.3037935e-04, 8.3863539e-01, 1.3859506e-01]],
       dtype=float32))

In [ ]:
print(classification_report(y_valid, np.argmax(y_hat, axis=1)))

              precision    recall  f1-score   support

           0       0.93      0.88      0.90      1900
           1       0.92      0.97      0.95      1900
           2       0.90      0.83      0.86      1900
           3       0.84      0.90      0.87      1900

    accuracy                           0.90      7600
   macro avg       0.90      0.90      0.89      7600
weighted avg       0.90      0.90      0.89      7600



In [ ]:
np.argmax(y_hat[:])

14040